In [3]:
import cv2
import mediapipe as mp
import numpy as np
import csv
import os
from IPython.display import display, HTML
import time
from PIL import ImageFont, ImageDraw, Image

def putText_korean(image, text, pos, font_path, font_size, color):
    # OpenCV 이미지를 PIL 이미지로 변환
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)
    # 폰트 로드
    font = ImageFont.truetype(font_path, font_size)
    # 텍스트 그리기
    draw.text(pos, text, font=font, fill=tuple(color[::-1]))
    # PIL 이미지를 다시 OpenCV 이미지로 변환하여 반환
    return cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

def collect_two_hands_data_notebook():
    # 1. 초기 설정
    mp_hands = mp.solutions.hands
    # NEW: max_num_hands를 2로 변경하여 두 손을 감지하도록 설정
    hands = mp_hands.Hands(
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils

    dataset_file = 'bsj_hand_landmark_dataset_two_hands.csv' # 새 파일 이름
    # 자음과 모음 라벨 목록
    consonant_labels = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    vowel_labels = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ','ㅣ','ㅘ','ㅚ', 'ㅙ', 'ㅝ','ㅟ','ㅞ','ㅢ']
    command_labels = ['shift', 'space', 'end'] # '쌍자음 만들기' 명령어만 남김
    labels = consonant_labels + vowel_labels + command_labels
    current_label_index = 0

    # NEW: CSV 파일 헤더를 두 손에 맞게 127개 열로 변경 (label + 126 coordinates)
    if not os.path.exists(dataset_file):
        with open(dataset_file, 'a', newline='', encoding = "UTF-8") as f:
            writer = csv.writer(f)
            # lh_ (왼손), rh_ (오른손) 접두사 사용
            header = ['label']
            for hand in ['lh', 'rh']:
                header += [f'{hand}_{i}_{axis}' for i in range(21) for axis in ['x', 'y', 'z']]
            writer.writerow(header)

    cap = cv2.VideoCapture(0)

    try:
        print("데이터 수집을 시작합니다. OpenCV 창을 활성화하고 키를 누르세요.")
        print("현재 라벨:", labels[current_label_index])

        capture_count = 0
        while cap.isOpened():
            success, image = cap.read()
            if not success: continue

            image = cv2.flip(image, 1)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image_rgb)
            
            # NEW: 두 손의 데이터를 담을 배열 초기화 (126개 공간을 0으로 채움)
            landmark_data = np.zeros(21 * 2 * 3) 
            detected_hands_count = 0

            if results.multi_hand_landmarks:
                detected_hands_count = len(results.multi_hand_landmarks)
                for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # handedness로 왼손/오른손 구분
                    handedness = results.multi_handedness[i].classification[0].label
                    
                    # 좌표 정규화
                    temp_hand_data = []
                    base_x, base_y, base_z = hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, hand_landmarks.landmark[0].z
                    for landmark in hand_landmarks.landmark:
                        temp_hand_data.extend([landmark.x - base_x, landmark.y - base_y, landmark.z - base_z])
                    
                    # 왼손은 앞부분(0-62), 오른손은 뒷부분(63-125)에 데이터 저장
                    if handedness == 'Left':
                        landmark_data[:21*3] = temp_hand_data
                    elif handedness == 'Right':
                        landmark_data[21*3:] = temp_hand_data

            current_label = labels[current_label_index]
            
            # --- 화면 안내 메시지 업데이트 ---
            #cv2.putText(image, f"Collecting: [{current_label}]", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            # 폰트 경로와 크기 변수 정의 
            font_path = "C:/Windows/Fonts/hmfmmuex.ttc"
            font_size = 30
            # putText_korean 함수로 변경
            image = putText_korean(image, f"Collecting: [{current_label}]", (10, 10), font_path, font_size, (0, 255, 0))
            cv2.putText(image, f"Detected Hands: {detected_hands_count}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
            if current_label in vowel_labels:
                cv2.putText(image, "Vowel: Show TWO hands", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 165, 255), 2, cv2.LINE_AA)
            else:
                 cv2.putText(image, "Consonant: Show ONE hand", (10, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 165, 0), 2, cv2.LINE_AA)

            cv2.putText(image, "s: Save", (480, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, "n: Next", (480, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, "q: Quit", (480, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
            
            cv2.imshow('Two-Hand Data Collection', image)

            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('n'):
                current_label_index = (current_label_index + 1) % len(labels)
                print(f"라벨 변경: {labels[current_label_index]}")
                capture_count = 0
            elif key == ord('s'):
                if detected_hands_count > 0:
                    # 모음 수집 시 손이 하나만 감지되면 경고 메시지 출력
                    if current_label in vowel_labels and detected_hands_count < 2:
                        print(f"경고: 모음 [{current_label}] 수집 시 두 손을 보여주세요. (현재 {detected_hands_count}개 감지)")
                    
                    with open(dataset_file, 'a', newline='') as f:
                        writer = csv.writer(f)
                        # np.array를 list로 변환하여 저장
                        row = [current_label] + landmark_data.tolist()
                        writer.writerow(row)
                    capture_count += 1
                    print(f"[{current_label}] 데이터 저장 완료! {capture_count}")
                else:
                    print("경고: 손이 감지되지 않아 저장할 수 없습니다.")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print(f"카메라가 해제되고 모든 창이 닫혔습니다.")
        print(f"데이터 수집이 완료되었습니다. '{dataset_file}' 파일을 확인하세요.")

# --- 함수 실행 ---
collect_two_hands_data_notebook()

데이터 수집을 시작합니다. OpenCV 창을 활성화하고 키를 누르세요.
현재 라벨: ㄱ
라벨 변경: ㄴ
라벨 변경: ㄷ
라벨 변경: ㄹ
라벨 변경: ㅁ
라벨 변경: ㅂ
라벨 변경: ㅅ
라벨 변경: ㅇ
라벨 변경: ㅈ
라벨 변경: ㅊ
라벨 변경: ㅋ
라벨 변경: ㅌ
라벨 변경: ㅍ
라벨 변경: ㅎ
라벨 변경: ㅏ
라벨 변경: ㅐ
라벨 변경: ㅑ
라벨 변경: ㅒ
라벨 변경: ㅓ
라벨 변경: ㅔ
라벨 변경: ㅕ
라벨 변경: ㅖ
라벨 변경: ㅗ
라벨 변경: ㅛ
라벨 변경: ㅜ
라벨 변경: ㅠ
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 1
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 2
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 3
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 4
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 5
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 6
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 7
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 8
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 9
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 10
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 11
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완료! 12
경고: 모음 [ㅠ] 수집 시 두 손을 보여주세요. (현재 1개 감지)
[ㅠ] 데이터 저장 완